In [1]:
from datetime import datetime, timedelta
from IPython.display import clear_output
import pandas_datareader.data as web
import yfinance as yf
import pandas as pd
import numpy as np
import pyautogui
import time

pd.options.display.float_format = '{:,.2f}'.format

# Downloads tickers and formatting data

### Fetching tickers from nasdaq and filtering out ETFs and stocks that we have a problem getting their prices

In [2]:
def get_all_tickers(etf=False):
    # Get a list of all available tickers from Yahoo Finance
    all_tickers = web.get_nasdaq_symbols()
    all_tickers = all_tickers[~all_tickers["Test Issue"]]  # Filter out stocks that we can't fetch their data for some reason
    
    if not etf:
        all_tickers = all_tickers[~all_tickers["ETF"]]  # Filter out ETFs
    tickers = all_tickers.index.tolist()
    return [str(ticker) for ticker in tickers if str(ticker).isalpha()]

### Formatting DataFrame

In [3]:
def get_info(df):
    high = df['High'].max()
    close = df['Close'].iloc[-1]
    pct_to_52 = ((high - close) / high) * 100
    volume = df["Volume"].iloc[-1]
    date_of_high = df['High'].idxmax()
    
    new_df = pd.DataFrame({'Close': close, '52wk High': high, "Percent from 52": pct_to_52, "Volume": volume, "doh": date_of_high, "Volume$": volume * close})

    return new_df

### Downloads data

In [4]:
def get_data(tickers_list, p="1y"):
    return yf.download(tickers_list, period=p)

In [5]:
s = time.time()
data = get_data(get_all_tickers())
data_cop = data.copy()
print(time.time() - s)

[*                      3%%                      ]  204 of 7141 completed

KeyboardInterrupt: 

[*                      3%%                      ]  205 of 7141 completed

In [18]:
df = get_info(data_cop)
cop = df.copy()
df

,Close,52wk High,Percent from 52,Volume,doh,Volume$
BKE,39.67,48.15,17.61,"490,900.00",2023-12-29,"19,474,002.10"
BME,42.60,43.85,2.85,"41,500.00",2023-02-01,"1,767,899.94"
CCTS,10.93,12.52,12.70,0.00,2023-10-31,0.00
EEIQ,1.91,3.30,42.12,"1,700.00",2023-05-24,"3,247.00"
FGBI,11.60,23.80,51.26,"17,400.00",2023-01-17,"201,840.01"
FLFVW,0.01,0.02,13.29,"10,666.00",2024-01-12,146.12
FMNB,13.35,14.90,10.40,"50,800.00",2023-01-13,"678,180.02"
LSXMB,30.71,42.00,26.88,200.00,2023-01-19,"6,142.00"
MKSI,101.31,114.15,11.25,"356,700.00",2023-02-02,"36,137,276.13"
RVSNW,0.05,0.05,0.00,150.00,2024-01-12,7.50


### Filtering out stocks that don't match 

In [14]:
df = cop.copy()
df = df.dropna(subset=['Close'])
vol_thresh = 1000
vold_thresh = 1000000
df = df[df['Volume'] >= vol_thresh]
df = df[df['Volume$'] >= vold_thresh]
time_thresh1 = datetime.now() - timedelta(days=45)  # Hit the high between 120 and 200 days ago (3 - 7 months)
time_thresh2 = datetime.now() - timedelta(days=245)  
df = df[(df["doh"] < time_thresh1) & (df["doh"] > time_thresh2)]
df.sort_values(by='Percent from 52', ascending=True).head(15)

,Close,52wk High,Percent from 52,Volume,doh,Volume$
VRTV,169.43,169.85,0.25,"122,600.00",2023-08-07,"20,772,117.10"
ROST,122.14,122.70,0.46,"1,443,600.00",2023-08-23,"176,321,303.12"
ARGO,29.94,30.13,0.63,"829,500.00",2023-04-06,"24,835,230.44"
NOW,609.25,614.36,0.83,"883,900.00",2023-07-19,"538,516,075.00"
ROP,503.20,508.90,1.12,"517,100.00",2023-09-20,"260,204,726.31"
DELL,71.97,72.82,1.17,"6,348,500.00",2023-09-11,"456,901,552.75"
RSG,154.65,156.65,1.28,"1,317,500.00",2023-08-01,"203,751,366.96"
BBDC,9.25,9.37,1.28,"300,600.00",2023-09-01,"2,780,550.00"
FSS,64.48,65.33,1.30,"324,400.00",2023-07-12,"20,917,313.09"
TWNK,33.30,33.74,1.30,"52,740,400.00",2023-09-13,"1,756,255,279.76"


# Testing out (manually backtesting)

In [32]:
def get_info_test(df):        
    days_back = 12
    
    _highs = df['High'][:-days_back].max()
    _closes = df['Close'].iloc[-days_back]
    _pct_to_52 = ((_highs - _closes) / _highs) * 100
    # _volume = np.nanmean(df["Volume"].iloc[-days_back])
    _date_of_high = df['High'][:-days_back].idxmax()
    
    highs = df['High'].max()
    date_of_high = df['High'].idxmax()
    closes = df['Close'].iloc[-1]
    volume = df["Volume"].iloc[-3:].mean()  # Calculates the average volume in the last 3 days
    pct_to_52 = ((highs - closes) / highs) * 100
    
    new_df = pd.DataFrame({'Old Close': _closes, 'Old 52wk High': _highs, "Old Percent from 52": _pct_to_52, "Volume": volume, "doh": _date_of_high, "Close": closes, "Prct252": pct_to_52, "high52": highs, "date of break": date_of_high})

    return new_df

In [33]:
df_test = get_info_test(data)
cop_test = df_test.copy()
cop_test

,Old Close,Old 52wk High,Old Percent from 52,Volume,doh,Close,Prct252,high52,date of break
A,127.62,160.26,20.37,"1,333,326.33",2022-12-13,126.87,20.83,160.26,2022-12-13
AA,33.38,58.24,42.68,"3,132,267.33",2022-08-26,33.80,41.96,58.24,2022-08-26
AAC,10.56,10.60,0.38,"21,163.33",2023-07-18,10.65,0.05,10.65,2023-07-31
AACG,1.39,2.64,47.35,"2,835.00",2022-08-18,1.32,50.00,2.64,2022-08-18
AACI,10.51,12.20,13.85,"8,276.50",2023-01-30,NaN,NaN,12.20,2023-01-30
...,...,...,...,...,...,...,...,...,...
ZVSA,0.20,25.00,99.21,"1,000,905.00",2022-12-12,0.17,99.33,25.00,2022-12-12
ZWS,27.33,31.77,13.98,"590,255.00",2022-08-18,30.21,4.91,31.77,2022-08-18
ZYME,7.87,10.80,27.13,"923,709.67",2023-01-19,7.16,33.66,10.80,2023-01-19
ZYNE,0.36,1.39,74.10,"96,968.00",2022-08-08,0.35,74.97,1.39,2022-08-08


### Filtering and sorting

In [7]:
df_test = cop_test.copy()
time_thresh = datetime.now() - timedelta(days=120)
break_time_thresh = datetime.now() - timedelta(days=2)  # How many days ago it broke
vol_thresh = 10000
# print(break_time_thresh)
df_test = df_test[df_test["Old 52wk High"] != df_test["high52"]]
df_test = df_test[df_test["doh"] < time_thresh]
df_test = df_test[df_test["date of break"] > break_time_thresh]
df_test["diff"] = datetime.now() - df_test["date of break"]
pd.options.display.float_format = '{:.2f}'.format
df_test.sort_values(by='Volume', ascending=False).head(20)

NameError: name 'cop_test' is not defined

# Random shit

In [9]:
all_tickers = web.get_nasdaq_symbols()
all_tickers = all_tickers[~all_tickers["ETF"] & ~all_tickers["Test Issue"]]
# all_tickers.loc["O"]
all_tickers

,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Symbol,,,,,,,,,,,
A,True,"Agilent Technologies, Inc. Common Stock",N,,False,100.00,False,NaN,A,A,False
AA,True,Alcoa Corporation Common Stock,N,,False,100.00,False,NaN,AA,AA,False
AAC,True,Ares Acquisition Corporation Class A Ordinary ...,N,,False,100.00,False,NaN,AAC,AAC,False
AAC.U,True,"Ares Acquisition Corporation Units, each consi...",N,,False,100.00,False,NaN,AAC.U,AAC=,False
AAC.W,True,Ares Acquisition Corporation Redeemable Warran...,N,,False,100.00,False,NaN,AAC.WS,AAC+,False
...,...,...,...,...,...,...,...,...,...,...,...
ZVSA,True,"ZyVersa Therapeutics, Inc. - Common Stock",Q,G,False,100.00,False,D,NaN,ZVSA,False
ZWS,True,Zurn Elkay Water Solutions Corporation Common ...,N,,False,100.00,False,NaN,ZWS,ZWS,False
ZYME,True,Zymeworks Inc. - Common Stock,Q,Q,False,100.00,False,N,NaN,ZYME,False


# Import to TradingView using pyautgui

In [6]:
def get_fit_tickers(df, num):
    return df.index[:num].tolist()


def add_to_tv(df):
    # Click plus (1773, 164)
    tickers = get_fit_tickers(df, 100)
    print(tickers)
    pyautogui.click(1773, 164)
    for ticker in tickers:
        pyautogui.write(ticker)
        pyautogui.press("enter")

# Finding stocks that broke 52wk high yesterday

### First DataFrame that contains necessary information

In [38]:
def parse_df(df, old_high=2, high_by="High"):
    # old_high- How many days ago it broke high (x or less)
    # high_by- by which OHLC to determine d
    high = df['High'].max()  # 52 wk high
    close = df['Close'].iloc[-1]  # Close of last candle
    pct_to_52 = ((high - close) / high) * 100  # Percent change to 52wk high
    volume = df["Volume"].iloc[-1]  # Volume of last candle
    old_doh = df['High'][:-old_high].idxmax()  # 52wk high before 2 days
    old_high = df["High"][:-old_high].max()
    # doh = df['High'].idxmax()  # 52wk high today
    # new_df = pd.DataFrame({'Close': close, '52wk High': high, "% to 52wk": pct_to_52, "Volume": volume, "Date of high": doh, "Old date of high": old_doh, "Volume$": volume * close})
    new_df = pd.DataFrame({'Close': close, 'old 52wk High': old_high, "Old date of high": old_doh, "Volume": volume, "Volume$": volume * close, "% to 52wk": pct_to_52})

    return new_df

### Filtering and sorting DataFrame

In [34]:
def filter_sort_df(df):
    df = df.copy()
    # df = df.dropna(subset=['Close'])
    vol_thresh = 10000  # Volume threshold
    vold_thresh = 500000  # Volume in dollar threshold, helps filtering out stocks that don't really have a high volume but due to low price they have a high volume
    time_thresh1 = datetime.now() - timedelta(days=45)# Hit the high between 120 and 200 days ago (3 - 7 months)
    # time_thresh1_b = datetime.now() - timedelta(days=60) 
    time_thresh2 = datetime.now() - timedelta(days=245)  
    # time_thresh2_b = datetime.now() - timedelta(days=200)
    df = df[df['Volume'] >= vol_thresh]
    df = df[df['Volume$'] >= vold_thresh]
    df = df[(df["Old date of high"] < time_thresh1) & (df["Old date of high"] > time_thresh2)]  # Old doh (last support) is between 90 to 200 days'
    df = df[df["Close"] > df["old 52wk High"]]
    
    return df.sort_values(by='% to 52wk', ascending=True)

In [39]:
full_df = parse_df(data, 1)
cop = full_df.copy()

df = filter_sort_df(cop)
df

,Close,old 52wk High,Old date of high,Volume,Volume$,% to 52wk
PANL,7.40,7.24,2023-06-21,"333,700.00","2,469,380.03",0.00
BPMC,68.45,68.00,2023-07-31,"284,800.00","19,494,559.13",0.10
NVO,105.45,104.00,2023-10-13,"4,598,700.00","484,932,900.97",0.23
USAP,17.02,16.99,2023-07-31,"59,100.00","1,005,882.03",1.05
PGTI,33.54,32.78,2023-10-10,"619,500.00","20,778,030.57",1.12
COIN,115.54,114.43,2023-07-14,"11,654,600.00","1,346,572,494.67",1.20
ANTX,17.24,17.00,2023-09-29,"36,100.00","622,363.99",2.87
YPF,16.84,16.19,2023-07-24,"11,069,800.00","186,415,433.69",5.39


In [10]:
add_to_tv(df)

['WFRD', 'UTI', 'FMX', 'AGYS', 'TIMB', 'WMT', 'FIF', 'CR', 'CHE', 'VST', 'ANET', 'PATI', 'GASS', 'PINS', 'CASI', 'AFYA', 'INFA', 'CNDB', 'CAPL', 'LII', 'ALSA', 'FTAI', 'TK']


In [14]:
cop.loc["URI"]

Close                            449.41
52wk High                        492.22
% to 52wk                          8.70
Volume                        24,350.00
Date of high        2023-08-07 00:00:00
Old date of high    2023-08-07 00:00:00
Volume$                   10,943,133.59
Name: URI, dtype: object

# Finding low 52

In [6]:
def parse_df_low(df, old_low=2, low_by="Low"):
    # old_low- How many days ago it broke low (x or less)
    # low_by- by which OHLC to determine d
    low = df['Low'].min()  # 52 wk low
    close = df['Close'].iloc[-1]  # Close of last candle
    pct_to_52 = ((low - close) / low) * 100  # Percent change to 52wk low
    volume = df["Volume"].iloc[-1]  # Volume of last candle
    old_doh = df['Low'][:-old_low].idxmin()  # 52wk low before 2 days
    doh = df['Low'].idxmin()  # 52wk low today
    new_df = pd.DataFrame({'Close': close, '52wk Low': low, "% to 52wk": pct_to_52, "Volume": volume, "Date of low": doh, "Old date of low": old_doh, "Volume$": volume * close})

    return new_df

def filter_sort_df_low(df):
    df = df.dropna(subset=['Close'])

    vol_thresh = 10000  # Volume threshold
    vold_thresh = 500000  # Volume in dollar threshold, helps filtering out stocks that don't really have a low volume but due to low price they have a low volume
    time_thresh1 = datetime.now() - timedelta(days=75)  # Hit the low between 120 and 200 days ago (3 - 7 months)
    time_thresh2 = datetime.now() - timedelta(days=200)  

    df = df[df['Volume'] >= vol_thresh]
    df = df[df['Volume$'] >= vold_thresh]
    df = df[(df["Old date of low"] < time_thresh1) & (df["Old date of low"] > time_thresh2)]  # Old doh (last support) is between 90 to 200 days'
    df = df[df["Old date of low"] != df["Date of low"]]
    
    # return df.sort_values(by='% to 52wk', ascending=True)
    return df.sort_values(by='Date of low', ascending=True)

In [7]:
full_df = parse_df_low(data, 12)
cop = full_df.copy()

df = filter_sort_df_low(cop)
df

,Close,52wk Low,% to 52wk,Volume,Date of low,Old date of low,Volume$
AHT,2.68,2.58,-3.88,"344,800.00",2023-08-22,2023-03-24,"924,064.02"
LC,6.85,6.23,-9.95,"1,428,700.00",2023-08-22,2023-05-04,"9,786,594.86"
SHBI,10.83,10.61,-2.07,"239,900.00",2023-08-22,2023-05-12,"2,598,116.98"
AROW,17.58,16.88,-4.15,"97,200.00",2023-08-23,2023-06-01,"1,708,775.99"
PTON,5.82,5.05,-15.25,"8,151,200.00",2023-08-23,2023-05-12,"47,439,985.40"
...,...,...,...,...,...,...,...
NXST,133.65,133.32,-0.25,"1,131,200.00",2023-09-07,2023-05-31,"151,184,873.10"
DG,123.72,123.31,-0.33,"5,281,400.00",2023-09-07,2023-06-12,"653,414,814.45"
OPI,5.85,5.81,-0.69,"2,365,600.00",2023-09-07,2023-05-02,"13,838,759.77"
EDBL,0.89,0.77,-15.89,"4,558,500.00",2023-09-07,2023-06-20,"4,057,064.93"


In [26]:
copcop = data.copy()
copcop = copcop.bfill()
copcop["Close"]

,A,AA,AAC,AACG,AACI,AACIU,AACIW,AACT,AADI,AAIC,...,ZUO,ZURA,ZURAW,ZVIA,ZVRA,ZVSA,ZWS,ZYME,ZYNE,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
2022-09-02,129.30,47.91,9.90,2.22,9.89,9.96,0.08,10.11,13.26,3.18,...,7.94,7.24,0.45,4.72,5.56,9.98,26.99,4.76,1.11,8.78
2022-09-06,129.30,47.91,9.90,2.22,9.89,9.96,0.08,10.11,13.26,3.18,...,7.89,7.24,0.45,4.72,5.56,9.98,26.99,4.76,1.11,8.78
2022-09-07,131.43,47.66,9.92,2.16,9.89,9.96,0.08,10.11,13.07,3.18,...,7.85,7.24,0.45,4.89,5.65,9.98,27.65,4.65,1.12,9.17
2022-09-08,135.19,48.86,9.92,2.15,9.89,9.96,0.08,10.11,12.87,3.20,...,7.92,7.24,0.45,5.05,5.93,9.98,27.98,5.07,1.18,9.30
2022-09-09,137.63,52.62,9.91,2.04,9.89,9.96,0.08,10.11,12.54,3.19,...,8.33,7.24,0.45,4.94,6.18,10.00,28.55,5.22,1.19,9.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-28,119.92,28.48,10.69,1.21,10.61,10.65,0.08,10.21,6.80,4.57,...,8.88,6.73,0.45,2.34,5.11,0.15,29.23,7.49,1.32,7.66
2023-08-29,122.02,29.61,10.69,1.24,10.61,10.65,0.08,10.21,6.71,4.62,...,8.91,6.79,0.45,2.49,5.31,0.14,29.39,7.53,1.32,7.71
2023-08-30,122.68,29.51,10.70,1.26,10.53,10.65,0.08,10.23,6.68,4.61,...,9.05,6.91,0.45,2.59,5.30,0.14,29.57,7.35,1.32,7.75
